# key words

In [ ]:
import requests

import pandas as pd

from bs4 import BeautifulSoup

from datetime import datetime

import pandas as pd

from textblob import TextBlob

import re
import nltk
from nltk.corpus import stopwords

In [ ]:
df600= pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/nba_author_credit/nba_600_credit_season_3.csv')
df1200= pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/nba_author_credit/nba_1200_credit_season_3.csv')
df1600= pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/nba_author_credit/nba_1600_credit_season_3.csv')


In [ ]:
def get_nba_content(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    content = soup.find("div", attrs={"class": "entry__content entry-content"})

    if content is None:
        return None

    main_body = content.find_all('p')
    text = " ".join([p.get_text(strip=True) for p in main_body])
    clean_text = text.replace('\n', ' ').replace('\xa0', '')
    return clean_text


def preprocess_text(text):
    text = text.lower()     # Convert text to lowercase
    
    text = re.sub(r"[^\w\s]", "", text) # Remove punctuation and special characters
    
    words = nltk.word_tokenize(text)     # Tokenize the text into words

    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words]
    
    preprocessed_text = " ".join(words)
    
    return preprocessed_text

def assess_nba_relevance(df):
    relevance_scores = []
    
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    
    for index, row in df.iterrows():
        content = get_nba_content(row['url'])
        player_name = row['player']
        if content is None:
            relevance_scores.append(0)
            continue
        preprocessed_content = preprocess_text(content)
        
        statement = player_name + " will be the MVP"
        statement_keywords = statement.lower().split()
        
        relevance_score = 0
        
        for keyword in statement_keywords:
            if keyword in preprocessed_content:
                relevance_score += 0.2
        
        # Check if player name is mentioned
        if player_name.lower() in preprocessed_content:
            relevance_score += 0.2
        
        relevance_scores.append(relevance_score)
    
    df['key_words_score'] = relevance_scores
    
    return df



